In [1]:
from portkey_ai import Portkey
from pydantic import BaseModel, Field
import random
import json
import os

portkey = Portkey(
    base_url="https://ai-gateway.apps.cloud.rt.nyu.edu/v1/",
    api_key=os.getenv("PORTKEY_API_KEY"),
)

#### Define a python function and specification for passing it to an LLM

In [2]:
def roll_dice(N: int) -> int:
    """Roll an N sided die"""
    return random.randint(1,N)

tools = [
    {
        "type": "function",
        "function": {
            "name": "roll_dice",
            "description": "Roll the special N sided dice and return the result",
            "parameters": {
                "type": "object",
                "properties": {
                    "num_sides": {
                        "type": "integer",
                        "description": "Number of sides for the die to roll"
                    }
                },
                "required": ["num_sides"]
            }
        }
    }
]

available_tools = {"roll_dice":roll_dice}

#### Initnal user prompt

In [3]:
messages = [
        {"role": "user", "content": "Roll a 10 sided die and check if the dice roll was valid. Explain your reasoning."}
]

In [4]:
completion = portkey.chat.completions.create(
    model="@vertexai/gemini-3-flash-preview",
    messages=messages,
    tools=tools,
)
messages.append(completion.choices[0].message)
print(completion.choices[0].message)

{
    "content": null,
    "role": "assistant",
    "function_call": null,
    "tool_calls": [
        {
            "id": "call_9RM3WYjsITc2woCT0fyfrpZb",
            "function": {
                "arguments": "{\"num_sides\":10}",
                "name": "roll_dice",
                "thought_signature": "CqkDAY89a1/yQwoj7rd5ndc5Q1bC4+QLvd/GIRIz225kMl+TbcJpZB/JlqAORzer+zQKr7XQCNQrSvilQNj8yIL9dl4SOWT1WoXCNK/TWFmibUS71BD2B9mDNzqpg2V8wbWYBxZ3DP6Lg5+NcP90LE9IQ7McjuV1uZ03QSj9NdpvgSOQGZ7SQq8MVSwO0AS2ZmxwKuoXUhLr/LMGjuGnXcYQdbyrUdPZeDYmFy2EpT5qnQgyZNLXSd3TGTDPbYekmfprrQAz5VN7g+wCJ00ri3+hM78XvVo3jDeZcJ9dUfNwH9U03xugzerEEzhUqnCB+KpbosMR1PWlY3wcJOaWyUDy8rXzGjgx28WuYcBl9H04AynwM/Oj/2vS77n3K8O124EAncNjB87ZJ3rKtUYn/sC9PLVEZ62xM7fmUWJERcVchDEHYwDmu18s1FldtzLB+qfGogad6YFS4DkU8UnIk7XpOZJcraIso2WuOMsl+xUD3EDtxCAB/vWUOmLpEXLtiZz+kVnpQxlp+9RfHbJT0fFcRPne31ldiRGQRhOfgDXQxjFu6Lm9Y1Dty7U="
            },
            "type": "function"
        }
    ],
    "refusal": null,
    "audio": null
}


### Extract the request from the LLM to call the tool and the argument to the tool

In [5]:
completion.choices[0].message.tool_calls[0].function

FunctionCall(arguments='{"num_sides":10}', name='roll_dice', thought_signature='CqkDAY89a1/yQwoj7rd5ndc5Q1bC4+QLvd/GIRIz225kMl+TbcJpZB/JlqAORzer+zQKr7XQCNQrSvilQNj8yIL9dl4SOWT1WoXCNK/TWFmibUS71BD2B9mDNzqpg2V8wbWYBxZ3DP6Lg5+NcP90LE9IQ7McjuV1uZ03QSj9NdpvgSOQGZ7SQq8MVSwO0AS2ZmxwKuoXUhLr/LMGjuGnXcYQdbyrUdPZeDYmFy2EpT5qnQgyZNLXSd3TGTDPbYekmfprrQAz5VN7g+wCJ00ri3+hM78XvVo3jDeZcJ9dUfNwH9U03xugzerEEzhUqnCB+KpbosMR1PWlY3wcJOaWyUDy8rXzGjgx28WuYcBl9H04AynwM/Oj/2vS77n3K8O124EAncNjB87ZJ3rKtUYn/sC9PLVEZ62xM7fmUWJERcVchDEHYwDmu18s1FldtzLB+qfGogad6YFS4DkU8UnIk7XpOZJcraIso2WuOMsl+xUD3EDtxCAB/vWUOmLpEXLtiZz+kVnpQxlp+9RfHbJT0fFcRPne31ldiRGQRhOfgDXQxjFu6Lm9Y1Dty7U=')

In [6]:
completion.choices[0].message.tool_calls[0].function

FunctionCall(arguments='{"num_sides":10}', name='roll_dice', thought_signature='CqkDAY89a1/yQwoj7rd5ndc5Q1bC4+QLvd/GIRIz225kMl+TbcJpZB/JlqAORzer+zQKr7XQCNQrSvilQNj8yIL9dl4SOWT1WoXCNK/TWFmibUS71BD2B9mDNzqpg2V8wbWYBxZ3DP6Lg5+NcP90LE9IQ7McjuV1uZ03QSj9NdpvgSOQGZ7SQq8MVSwO0AS2ZmxwKuoXUhLr/LMGjuGnXcYQdbyrUdPZeDYmFy2EpT5qnQgyZNLXSd3TGTDPbYekmfprrQAz5VN7g+wCJ00ri3+hM78XvVo3jDeZcJ9dUfNwH9U03xugzerEEzhUqnCB+KpbosMR1PWlY3wcJOaWyUDy8rXzGjgx28WuYcBl9H04AynwM/Oj/2vS77n3K8O124EAncNjB87ZJ3rKtUYn/sC9PLVEZ62xM7fmUWJERcVchDEHYwDmu18s1FldtzLB+qfGogad6YFS4DkU8UnIk7XpOZJcraIso2WuOMsl+xUD3EDtxCAB/vWUOmLpEXLtiZz+kVnpQxlp+9RfHbJT0fFcRPne31ldiRGQRhOfgDXQxjFu6Lm9Y1Dty7U=')

In [7]:
function_to_call = available_tools[completion.choices[0].message.tool_calls[0].function.name] 

In [8]:
dice_sides_to_roll = json.loads(completion.choices[0].message.tool_calls[0].function.arguments)["num_sides"]

In [9]:
dice_roll_result = function_to_call(dice_sides_to_roll)

### Prepare a message to send to the LLM with the results of the tool call

In [10]:
result_message = {
    "role": "tool",
    "tool_call_id": completion.choices[0].message.tool_calls[0].id,
    "name": completion.choices[0].message.tool_calls[0].function.name,
    "content": json.dumps(dice_roll_result)
}


In [11]:
messages.append(result_message)

In [12]:
messages

[{'role': 'user',
  'content': 'Roll a 10 sided die and check if the dice roll was valid. Explain your reasoning.'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_9RM3WYjsITc2woCT0fyfrpZb', function=FunctionCall(arguments='{"num_sides":10}', name='roll_dice', thought_signature='CqkDAY89a1/yQwoj7rd5ndc5Q1bC4+QLvd/GIRIz225kMl+TbcJpZB/JlqAORzer+zQKr7XQCNQrSvilQNj8yIL9dl4SOWT1WoXCNK/TWFmibUS71BD2B9mDNzqpg2V8wbWYBxZ3DP6Lg5+NcP90LE9IQ7McjuV1uZ03QSj9NdpvgSOQGZ7SQq8MVSwO0AS2ZmxwKuoXUhLr/LMGjuGnXcYQdbyrUdPZeDYmFy2EpT5qnQgyZNLXSd3TGTDPbYekmfprrQAz5VN7g+wCJ00ri3+hM78XvVo3jDeZcJ9dUfNwH9U03xugzerEEzhUqnCB+KpbosMR1PWlY3wcJOaWyUDy8rXzGjgx28WuYcBl9H04AynwM/Oj/2vS77n3K8O124EAncNjB87ZJ3rKtUYn/sC9PLVEZ62xM7fmUWJERcVchDEHYwDmu18s1FldtzLB+qfGogad6YFS4DkU8UnIk7XpOZJcraIso2WuOMsl+xUD3EDtxCAB/vWUOmLpEXLtiZz+kVnpQxlp+9RfHbJT0fFcRPne31ldiRGQRhOfgDXQxjFu6Lm9Y1Dty7U='), type='function')], refusal=None, audio=None),
 {'role': 'tool',
 

### Pass all the messages, the initial prompt, the first response from the LLM requesting tool use and finally the result of the tool use!

In [13]:
completion = portkey.chat.completions.create(
    model="@vertexai/gemini-3-flash-preview",
    messages=messages,
    tools=tools,
)
print(completion.choices[0].message)

{
    "content": "The result of the 10-sided die roll is **7**.\n\n### Validity Check and Reasoning:\nThe dice roll is **valid**. \n\n**Reasoning:**\n1.  **Range:** A 10-sided die (d10) produces results in the inclusive range of 1 to 10.\n2.  **Result:** The rolled value is 7.\n3.  **Comparison:** Since 7 is an integer greater than or equal to 1 and less than or equal to 10 ($1 \\le 7 \\le 10$), it falls within the expected mathematical parameters for this die.",
    "role": "assistant",
    "function_call": null,
    "tool_calls": null,
    "refusal": null,
    "audio": null,
    "content_blocks": [
        {
            "type": "text",
            "text": "The result of the 10-sided die roll is **7**.\n\n### Validity Check and Reasoning:\nThe dice roll is **valid**. \n\n**Reasoning:**\n1.  **Range:** A 10-sided die (d10) produces results in the inclusive range of 1 to 10.\n2.  **Result:** The rolled value is 7.\n3.  **Comparison:** Since 7 is an integer greater than or equal to 1 and